In [ ]:
import requests

response = requests.get('http://127.0.0.1:5000/hello')
print(response.text)

data = {'a': 1, 'b': 2, 'c' : 3}
response = requests.post('http://127.0.0.1:5000/api', json=data)
print(response.json())

## Fetch Data through API

In [10]:
import pandas as pd
import requests
from io import StringIO
import json

# URL for the CSV file
url = 'https://reg.bom.gov.au/climate/dwo/202403/text/IDCJDW3050.202403.csv'

# Send HTTP request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Skip the initial 7 lines of metadata, then read the content into a pandas DataFrame
    data = pd.read_csv(StringIO(response.text), skiprows=7)
    data = data.drop(columns=['Unnamed: 0'])
    
    # Convert the DataFrame to a JSON string
    json_data = data.to_json(orient='records', lines=True)
    
    print(json_data)
    
    # Save the JSON data to a file
    with open('data.json', 'w') as file:
        file.write(json_data)
    
else:
    print(f"Failed to retrieve data: {response.status_code}")


{"Date":"2024-03-1","Minimum temperature (\u00b0C)":14.4,"Maximum temperature (\u00b0C)":28.1,"Rainfall (mm)":0.0,"Evaporation (mm)":5.2,"Sunshine (hours)":11.0,"Direction of maximum wind gust ":"SSW","Speed of maximum wind gust (km\/h)":37,"Time of maximum wind gust":"16:13","9am Temperature (\u00b0C)":18.0,"9am relative humidity (%)":72,"9am cloud amount (oktas)":3,"9am wind direction":"NE","9am wind speed (km\/h)":"2","9am MSL pressure (hPa)":1015.1,"3pm Temperature (\u00b0C)":26.5,"3pm relative humidity (%)":49,"3pm cloud amount (oktas)":3,"3pm wind direction":"SSW","3pm wind speed (km\/h)":9,"3pm MSL pressure (hPa)":1011.7}
{"Date":"2024-03-2","Minimum temperature (\u00b0C)":15.9,"Maximum temperature (\u00b0C)":20.7,"Rainfall (mm)":0.0,"Evaporation (mm)":5.6,"Sunshine (hours)":8.1,"Direction of maximum wind gust ":"S","Speed of maximum wind gust (km\/h)":35,"Time of maximum wind gust":"13:08","9am Temperature (\u00b0C)":18.7,"9am relative humidity (%)":67,"9am cloud amount (oktas)

## Inserting Data into Elasticsearch

In [6]:
import csv
with open('accident_location.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
            print(row)
            

{'ACCIDENT_NO': 'T20220000059', 'LATITUDE': '-37.758575', 'LONGITUDE': '145.316211'}
{'ACCIDENT_NO': 'T20220002574', 'LATITUDE': '-37.799725', 'LONGITUDE': '144.789329'}
{'ACCIDENT_NO': 'T20220000011', 'LATITUDE': '-37.844734', 'LONGITUDE': '144.788766'}
{'ACCIDENT_NO': 'T20220000041', 'LATITUDE': '-37.269035', 'LONGITUDE': '145.458471'}
{'ACCIDENT_NO': 'T20220001286', 'LATITUDE': '-38.338641', 'LONGITUDE': '144.301917'}
{'ACCIDENT_NO': 'T20220000111', 'LATITUDE': '-37.84744', 'LONGITUDE': '144.67893'}
{'ACCIDENT_NO': 'T20220000039', 'LATITUDE': '-37.782585', 'LONGITUDE': '145.118045'}
{'ACCIDENT_NO': 'T20220003468', 'LATITUDE': '-37.907053', 'LONGITUDE': '144.631062'}
{'ACCIDENT_NO': 'T20220000014', 'LATITUDE': '-38.104797', 'LONGITUDE': '145.178091'}
{'ACCIDENT_NO': 'T20220000009', 'LATITUDE': '-38.247725', 'LONGITUDE': '145.826305'}
{'ACCIDENT_NO': 'T20220002291', 'LATITUDE': '-37.982174', 'LONGITUDE': '145.071351'}
{'ACCIDENT_NO': 'T20220000054', 'LATITUDE': '-37.689501', 'LONGITUD

### Create the Index with the Mapping

In [12]:
from elasticsearch import Elasticsearch

# Connect to Elasticsearch
es = Elasticsearch(['http://localhost:9200'])

# Define the mapping for the index
mapping = {
    "mappings": {
        "properties": {
            "Date": {"type": "date", "format": "yyyy-MM-dd"},
            "Minimum temperature (°C)": {"type": "float"},
            "Maximum temperature (°C)": {"type": "float"},
            "Rainfall (mm)": {"type": "float"},
            "Evaporation (mm)": {"type": "float"},
            "Sunshine (hours)": {"type": "float"},
            "Direction of maximum wind gust": {"type": "keyword"},
            "Speed of maximum wind gust (km/h)": {"type": "float"},
            "Time of maximum wind gust": {"type": "text"},  # or 'date_hour_minute' if formatted correctly
            "9am Temperature (°C)": {"type": "float"},
            "9am relative humidity (%)": {"type": "integer"},
            "9am cloud amount (oktas)": {"type": "integer"},
            "9am wind direction": {"type": "keyword"},
            "9am wind speed (km/h)": {"type": "float"},
            "9am MSL pressure (hPa)": {"type": "float"},
            "3pm Temperature (°C)": {"type": "float"},
            "3pm relative humidity (%)": {"type": "integer"},
            "3pm cloud amount (oktas)": {"type": "integer"},
            "3pm wind direction": {"type": "keyword"},
            "3pm wind speed (km/h)": {"type": "float"},
            "3pm MSL pressure (hPa)": {"type": "float"}
        }
    }
}

# Index name
index_name = "weather"

# Create the index with the defined mapping
# es.indices.create(index=index_name, body=mapping, ignore=400)  # ignore 400 to not raise exception if index already exists


curl -XPUT -k 'https://127.0.0.1:9200/weather' \
  --header 'Content-Type: application/json' \
  --data '{
    "settings": {
        "index": {
            "number_of_shards": 3,
            "number_of_replicas": 1
        }
    },
    "mappings": {
        "properties": {
            "Date": {"type": "date", "format": "yyyy-MM-d"},
            "Minimum temperature (°C)": {"type": "float"},
            "Maximum temperature (°C)": {"type": "float"},
            "Rainfall (mm)": {"type": "float"},
            "Evaporation (mm)": {"type": "float"},
            "Sunshine (hours)": {"type": "float"},
            "Speed of maximum wind gust (km/h)": {"type": "float"},
            "Time of maximum wind gust": {"type": "text"},
            "9am Temperature (°C)": {"type": "float"},
            "9am relative humidity (%)": {"type": "integer"},
            "9am cloud amount (oktas)": {"type": "integer"},
            "9am wind speed (km/h)": {"type": "integer"},
            "9am MSL pressure (hPa)": {"type": "float"},
            "3pm Temperature (°C)": {"type": "float"},
            "3pm relative humidity (%)": {"type": "integer"},
            "3pm cloud amount (oktas)": {"type": "integer"},
            "3pm wind speed (km/h)": {"type": "integer"},
            "3pm MSL pressure (hPa)": {"type": "float"}
        }
    }
  }' \
  --user 'elastic:elastic' | jq '.'


### Insert Data into the Index

In [ ]:
from elasticsearch import Elasticsearch

# Assuming Elasticsearch and index_name are properly set up
es = Elasticsearch(['http://localhost:9200'], http_auth=('elastic', 'elastic'), verify_certs=False)
index_name = 'weather'

with open('data.json', 'r') as file:
    data_df = pd.read_json(file, lines=True)

record = data_df.to_dict(orient='records')[0]
response = es.index(index=index_name, document=record)
print("Document ID:", response['_id'])
print(response)
# Index each record in the DataFrame
# for record in data_df.to_dict(orient='records'):
#     print(record)
    # es.index(index=index_name, body=record)

: 